# Multiple planes to HDF

In [1]:
from h5rdmtoolbox import x2hdf, tutorial, H5PIV

c:\users\da4323\documents\programming\github\h5rdmtoolbox\h5rdmtoolbox\h5wrapper\h5piv.py:851: SpecialDatasetRegistrationWarning: registration of accessor <class 'h5rdmtoolbox.h5wrapper.h5piv.PIVDisplacementDataset'> under name 'DisplacementVector' for type <class 'h5rdmtoolbox.h5wrapper.h5piv.H5PIVGroup'> is overriding a preexisting attribute with the same name.
  class PIVDisplacementDataset(DisplacementDataset):


To convert a multiple planes to a single HDF file (multiplane-file), you must provide the plane folders containing the netCDF4 files.<br>
**For building a multiplane-file, there are a few scenarios to be aware of which will result in different setups of the HDF file structure!**

## Planes with unequal recording frequency

If all planes (in this tutorials two planes) are recorded with **different** measurement frequencies (laser may be triggered leading to slight time differences while average frequency is similar!), then data variables cannot be packed into a **single array** because the time vector is different for each plane. Thus, groups are generated for each plane where each data variable has shape (`nt`, `ny`, `nx`). Here, the number of time steps (`nt`) could also be different for each plane. However, assuming, recording physically/conceptionally belongs to the same "case" it is still reasonable to pack all data in one file. **The time-averaged data is in one data array** having the shape (`nz`, `ny`, `nx`):

In [2]:
plane0_dir, plane1_dir, plane2_dir = tutorial.PIVview.get_multiplane_directories()

In [3]:
piv_mplane_uef = x2hdf.PIVMultiPlane([plane0_dir, plane1_dir], time_information=[5, 5.2])  # force different planes by passing different meas. freqs.
piv_mplane_uef.convert()
print(f'conversion time: {piv_mplane_uef.conversion_time}, generated file: {piv_mplane_uef.hdf_filename}')
for plane in piv_mplane_uef.planes:
    print(f'plane {plane.name.stem}: {plane.nc}')
print(f'hdf file size: {piv_mplane_uef.hdf_filename.stat().st_size} Bytes ({piv_mplane_uef.hdf_filename.stat().st_size/1000} KB)')

  0%|          | 0/2 [00:00<?, ? snapshots/s]

  0%|          | 0/2 [00:00<?, ? snapshots/s]

conversion time: 0.33574938774108887, generated file: C:\Users\da4323\AppData\Local\h5rdmtoolbox\h5rdmtoolbox\tmp\tmp65\mplane.hdf
plane 0: 3 nc files -> total size: 132552 Bytes (133 KB) 
plane 1: 3 nc files -> total size: 132552 Bytes (133 KB) 
hdf file size: 520847 Bytes (520.847 KB)


In [4]:
with H5PIV(piv_mplane_uef.hdf_filename, 'r') as h5m:
    h5m.dump()

c:\users\da4323\documents\programming\github\h5rdmtoolbox\h5rdmtoolbox\h5wrapper\h5file.py:1639: StandardizedNameTableWarning: Unable to write standard name table to file. It will be effective anyhow but if the file is reloaded later the choice is forgotten.
  warnings.warn(


## Planes with equal recording frequency

If all planes (in this tutorials two planes) are recorded with the same measurement frequency, then data variables can be packed into a **single array** with shape (`nz`, `nt`, `ny`, `nx`), which in this tutorial is (2, 5, 110, 127). The time-averaged data has shape (`nz`, `ny`, `nx`)

In [5]:
piv_mplane_ef = x2hdf.PIVMultiPlane([plane0_dir, plane1_dir], time_information=[5, 5])
piv_mplane_ef.convert()
print(f'conversion time: {piv_mplane_ef.conversion_time}, generated file: {piv_mplane_ef.hdf_filename}')

  0%|          | 0/2 [00:00<?, ? snapshots/s]

  0%|          | 0/2 [00:00<?, ? snapshots/s]

conversion time: 0.2762160301208496, generated file: C:\Users\da4323\AppData\Local\h5rdmtoolbox\h5rdmtoolbox\tmp\tmp65\mplane.hdf


In [6]:
with H5PIV(piv_mplane_ef.hdf_filename, 'r') as h5m:
    h5m.dump()

---
## Additional Functionalities of the package

1. computing additional (intermediate) measurement planes
2. combining the intermediate planes with the measured planes to a single new, "virtual" multi-plane file

Final, folder/file structure will look like:

Now let's generate **intermediate** planes by interpolating them.  The following builds 6 equally spaced z-planes. As this includes the outer two (measured) planes, only 4 planes will be interpolated:

In [7]:
with H5PIV(piv_mplane_ef.hdf_filename, 'r+') as h5m:
    h5m.dump()
    h5m.z[:] = [0, 1]

In [8]:
ipfile = piv_mplane_ef.compute_intermediate_plane(6)

Reduced number planes to generate from 6 to 4 as some of the input planes already exist from the measurement.
interpolating...
> /timeAverages/dudx: 1.32 ms
> /timeAverages/dudy: 1.58 ms
> /timeAverages/dvdx: 1.45 ms
> /timeAverages/dvdy: 1.13 ms
> /timeAverages/dx: 1.13 ms
> /timeAverages/dy: 1.04 ms
> /timeAverages/piv_correlation_coeff: 1.03 ms
> /timeAverages/piv_peak1_corr: 1.13 ms
> /timeAverages/piv_peak1_dx: 1.24 ms
> /timeAverages/piv_peak1_dy: 1.20 ms
> /timeAverages/piv_peak2_corr: 1.42 ms
> /timeAverages/piv_peak2_dx: 1.60 ms
> /timeAverages/piv_peak2_dy: 2.03 ms
> /timeAverages/piv_peak3_corr: 1.24 ms
> /timeAverages/piv_peak3_dx: 1.09 ms
> /timeAverages/piv_peak3_dy: 1.14 ms
> /timeAverages/piv_snr_data: 1.23 ms
> /timeAverages/u: 1.17 ms
> /timeAverages/v: 1.08 ms
> /timeAverages/velmag: 1.14 ms
...done.


Note that the original measurement file had two planes. The upper method generated actually not 6 but 4 planes because the outer ones already exist in the measurement file

As the two files are separated, we now create a multi-plane file that has all original and interpolated planes in one file. We call such a case a **virtual case**

In [9]:
vcase_name = piv_mplane_ef.build_virtual_mplane_file(ipfile)  # pass the just created file name

---
## Converting a PIVview "average dat" file
There are three "average" files in pivview "\*reyn.dat", "\*rms.dat", "\*avg.dat". The latter ("\*avg.dat") **must** be exist for the following method!

In [10]:
from h5rdmtoolbox.x2hdf.piv2hdf.pivview.plane import build_plane_hdf_from_average_dat_files

ModuleNotFoundError: No module named 'h5rdmtoolbox.x2hdf.piv2hdf'

In [ ]:
avg_filename = tutorial.PIVview.get_avg_file()
hdf_file = build_plane_hdf_from_average_dat_files(avg_filename, None, None,
                                                  target=str(avg_filename).replace('.dat', '.hdf'))  # reyn and rms file don't exist, so pass None!!!

In [ ]:
with H5PIV(hdf_file, 'r') as h5:
    h5.dump()